In [ ]:
from neuprint import Client
from neuprint import fetch_neurons
from neuprint import NeuronCriteria as NC
from neuprint import SynapseCriteria as SC


with open('auth_token.txt', 'r') as f:
    token = f.read()

c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

import pandas as pd

from matplotlib import rcParams

rcParams['font.family'] = 'Arial'
rcParams['font.size'] = 9


xl = pd.read_excel('Putative IDs.xlsx')
xl.columns = xl.loc[0,:]
xl.drop(0, inplace=True)
# drop first column
xl.drop(xl.columns[0], axis=1, inplace=True)

putative_cdn =  xl['CDN'].loc[~xl['CDN'].isnull()].values.astype(int)

pcdn = fetch_neurons(
    NC(
        bodyId=putative_cdn,
    ),
    client=c
)[0]

In [ ]:
import bokeh.palettes
from bokeh.models import Range1d
from bokeh.plotting import figure, show, output_notebook
import numpy as np
from neuprint import fetch_synapses
from neuprint import SynapseCriteria as SC
output_notebook()

pcdn_gab = pcdn[pcdn['ntGabaProb'] >= 0.7]

from neuprint import fetch_adjacencies

# _, pcdn_conn = fetch_adjacencies(
#     targets = NC(
#         bodyId=pcdn_gab['bodyId'].values,
#     ),
#     min_total_weight = 5,
# )

_, pcdn_conn = fetch_adjacencies(
    targets = NC(
        bodyId=pcdn['bodyId'].values,
    ),
    min_total_weight = 5,
)

_, pcdn_conn_out = fetch_adjacencies(
    sources = NC(
        bodyId=pcdn_gab['bodyId'].values,
    ),
    min_total_weight = 5,
)

# _, pcdn_conn_out = fetch_adjacencies(
#     sources = NC(
#         bodyId=pcdn['bodyId'].values,
#     ),
#     min_total_weight = 5,
# )


import seaborn as sns
import matplotlib.pyplot as plt

matrix = pcdn_conn_out.pivot_table(index='bodyId_pre', columns='bodyId_post', values='weight', aggfunc='sum', fill_value=0)
post_mat = pcdn_conn.pivot_table(index='bodyId_post', columns='bodyId_pre', values='weight', aggfunc='sum', fill_value=0)


def cosine_similarity(matrix):
    """
    Calculate the cosine similarity for a matrix.
    """
    from sklearn.metrics.pairwise import cosine_similarity

    # Calculate cosine similarity between rows
    cosine_sim_matrix = cosine_similarity(matrix)

    # Create a DataFrame with cosine similarity matrix
    cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

    return cosine_sim_df


# Calculate cosine similarity between rows
cosine_sim_matrix = cosine_similarity(matrix)

# Create a DataFrame with cosine similarity matrix
cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=matrix.index, columns=matrix.index)

cosine_sim_post = cosine_similarity(post_mat)
cosine_sim_post_df = pd.DataFrame(cosine_sim_post, index=post_mat.index, columns=post_mat.index)

# Order the sorted matrix with hierarchical clustering
from scipy.cluster.hierarchy import linkage, dendrogram

# Calculate the linkage matrix
linkage_matrix = linkage(cosine_sim_post_df, 'average')

# Plot the dendrogram
plt.figure(figsize=(10,10))
r = dendrogram(linkage_matrix, labels=cosine_sim_post_df.index, orientation='left', leaf_font_size=10, no_plot=True)

sns.heatmap(
    cosine_sim_post_df.loc[r['ivl']][r['ivl']],
    #cosine_sim_df.loc[r['ivl']][r['ivl']],
    cmap='Blues',
    square=True,
    xticklabels=False,
    yticklabels=False,
    cbar_kws={'label': 'Cosine similarity of output vectors'},
    vmin=0,
    vmax=1,
)

plt.gcf().set_size_inches(10,10)

#plt.gcf().savefig('All CDN morphology inputs cosine similarity.svg')

In [ ]:
sns.heatmap(
    #post_mat.loc[r['ivl']],
    matrix.loc[r['ivl']],
    #cosine_sim_df.loc[sorted_rows][sorted_rows] * cosine_sim_post_df.loc[sorted_rows][sorted_rows],
    cmap='Reds',
    square=False,
    xticklabels=False,
    yticklabels=False,
    cbar_kws={'label': 'Synapse count'},
    vmin=0,
    vmax = 50,
)

plt.gca().set_xlabel('')
plt.gca().set_ylabel('')
plt.gcf().set_size_inches(20,5)

#plt.gcf().savefig('CDN output synapse count.png')

In [ ]:
cluster1 = [15860, 17405, 23216, 16787, 17301]
cluster2 = [14640, 34562, 16090, 18039, 101455, 16410, 101860]
cluster2_with_nongaba = [11394, 14097,22477, 155389, 16647, 16781, 14640, 34562, 16090, 18039, 101455, 16410, 101860]
cluster3 = [21196, 21732, 14457, 18011, 15861, 16676]
cluster4 = [13856, 14879, 14593, 16934, 14933, 15461]
cluster5 = [17452, 23036, 19902, 14371, 16252, 13174, 13605]

clusters = [cluster1, cluster2, cluster3, cluster4, cluster5]

In [ ]:
rcParams['font.size'] = 12

def plot_reciprocity(
    cluster : list[int],
    ax = None,
    pool : bool = True,
    **plot_kwargs,
):

    _, cluster_out_conns = fetch_adjacencies(
        sources = NC(
            bodyId = cluster,
        )
    )

    _, cluster_in_conns = fetch_adjacencies(
        sources = NC(
            bodyId = cluster_out_conns['bodyId_post'].unique(),
        ),
        targets = NC(
            bodyId = cluster,
        )
    )

    cluster_out_conns

    if pool:
        merged_cluster = cluster_out_conns.groupby('bodyId_post').agg({'weight' : 'sum'}).merge(
            cluster_in_conns.groupby('bodyId_pre').agg({'weight' : 'sum'}),
            left_index=True,
            right_index=True,
            suffixes=('_in', '_out'),
        )

    else:
        merged_cluster = cluster_out_conns.merge(
            cluster_in_conns,
            left_on='bodyId_post',
            right_on='bodyId_pre',
            suffixes=('_in', '_out'),
        )

    #merged_cluster = merged_cluster.groupby(['bodyId_post_out']).agg({'weight_out': 'sum', 'weight_in': 'sum'}).reset_index()
    #print(merged_cluster)
    if ax is None:
        fig, ax = plt.subplots()

    plot_kwargs = {
        'alpha': 0.5,
        'marker' : 'o',
        'linewidth' : 0,
        **plot_kwargs,
    }

    ax.scatter(
        merged_cluster['weight_in'],
        merged_cluster['weight_out'],
        **plot_kwargs,
    )

    ax.set_xlabel('Input synapse count')
    ax.set_ylabel('Output synapse count')

    # compute spearman correlation
    from scipy.stats import spearmanr
    rho, p = spearmanr(merged_cluster['weight_in'], merged_cluster['weight_out'])
    #ax.set_title(f'Spearman rho: {rho:.2f}, p={p:.2e}')
    print(f'Spearman rho: {rho:.2f}, p={p:.2e}')
    return ax

fig, axs = plt.subplots(1,5, figsize=(15,3))

for cluster, ax in zip(clusters, axs.flatten()):
    plot_reciprocity(cluster, ax=ax, color = '#000000')
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    #ax.set_aspect('equal')
                     


plt.tight_layout()
fig.savefig('CDN cluster reciprocity clusterwide.png')

fig, axs = plt.subplots(1,5, figsize=(15,3))

for cluster, ax in zip(clusters, axs.flatten()):
    plot_reciprocity(cluster, ax=ax, color = '#000000', pool=False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    #ax.set_aspect('equal')

plt.tight_layout()
fig.savefig('CDN cluster reciprocity cellwise.png')

In [ ]:
CINNABAR = '#db544b'
LAPIS = '#2d66a5'
BYZANTINE = '#D342BE'
SHAMROCK = '#33A358'

roi_mesh = c.fetch_roi_mesh('ANm')

from bokeh.plotting import figure, show, output_notebook
output_notebook()
def plot_neurons(
        source_df, idx : int,
        flattened_ax : int = 1,
        on_p = None,
        weight : float = 1,
        color : str = '#000000',
        plot_synapses : bool = False,
        roi_mesh = c.fetch_roi_mesh('ANm'),
        ):

        skelee = c.fetch_skeleton(source_df.iloc[idx]['bodyId'])

        skelee.radius = skelee.radius.astype(float)/(20)
        # Join parent/child nodes for plotting as line segments below.
        # (Using each row's 'link' (parent) ID, find the row with matching rowId.)
        skelee = skelee.merge(skelee, 'inner',
                                left_on=['link'],
                                right_on=['rowId'],
                                suffixes=['_child', '_parent'])
        if on_p is None:
            p = figure(
                match_aspect = True
            )
            p.y_range.flipped = False
        else:
            p = on_p

        p.toolbar.logo = None
        p.toolbar_location = None

        axes = [x for x in range(3) if x != flattened_ax]
        if not roi_mesh is None:
            if not isinstance(roi_mesh, list):
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh).split('\\n') if row.startswith('v')
                ])
            else:
                verts = np.array([
                    [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                    for row in str(roi_mesh[0]).split('\\n') if row.startswith('v')
                ])
                for mesh in roi_mesh:
                    verts = np.concatenate((
                        verts,
                        np.array([
                            [float(row.split(' ')[axes[0]+1]),float(row.split(' ')[axes[1]+1])]
                            for row in str(mesh).split('\\n') if row.startswith('v')
                        ])
                    ))

            speckle_size = 1.0
            p.scatter(verts[:,0],verts[:,1],size=speckle_size,line_alpha=0.0, fill_color = (0,0,0), fill_alpha = 0.5)

        str_labels = ['x', 'y', 'z']

        as_str = [str_labels[i] for i in axes]

        # Plot skeleton segments (in 2D)
        p.segment(
            x0=f'{as_str[0]}_child', x1=f'{as_str[0]}_parent',
            y0=f'{as_str[1]}_child', y1=f'{as_str[1]}_parent',
            line_width = 'radius_parent',
            alpha = 0.7*weight,
            color = color,
            source=skelee
        )

        if plot_synapses:
            syns = fetch_synapses(NC(bodyId=source_df.iloc[idx]['bodyId']))
            pre = syns.loc[syns['type'] == 'pre']
            p.scatter(
                pre[as_str[0]], pre[as_str[1]],
                size = 3, fill_color=BYZANTINE,
                fill_alpha = 0.9, line_color = None,
                line_alpha = 0.0, line_width=0,
            )
            
            post = syns.loc[syns['type'] == 'post']
            
            p.scatter(
                post[as_str[0]], post[as_str[1]],
                size = 3, fill_color=SHAMROCK,
                fill_alpha = 0.9, line_color=None,
                line_width = 0,
            )        

        p.renderers.insert(0,p.renderers.pop())

        if not (roi_mesh is None or isinstance(roi_mesh, list)):
            lims = {
                'x' : Range1d(18000,30000),
                'y' : Range1d(0,20000),
                'z' : Range1d(0,20000)
            }

            if as_str[0] in lims:
                p.x_range = lims[as_str[0]]
            if as_str[1] in lims:
                p.y_range = lims[as_str[1]]

        #p.y_range = Range1d(0,20000)
        #p.x_range = Range1d(18000,30000)
        p.xgrid.visible = False
        p.ygrid.visible = False
        p.axis.visible = False
        return p

from bokeh.plotting import gridplot

from bokeh.io import export_png

In [ ]:

cdn_df = fetch_neurons(
    NC(
        bodyId = cluster2
    ),
    client=c
)[0]

# color palette
pal = bokeh.palettes.Set1_7

# make a grid
on_ps = [
     plot_neurons(cdn_df, 0, 0, color = pal[0]),
     plot_neurons(cdn_df, 0, 1, color = pal[0]),
     plot_neurons(cdn_df, 0, 2, color = pal[0]),
]
for idx in range(1, len(cdn_df)):
     plot_neurons(cdn_df, idx, 0, on_p = on_ps[0], color = pal[idx%len(pal)]) 
     plot_neurons(cdn_df, idx, 1, on_p = on_ps[1], color = pal[idx%len(pal)])
     plot_neurons(cdn_df, idx, 2, on_p = on_ps[2], color = pal[idx%len(pal)])

grid_time = gridplot([on_ps])

show(grid_time)
#export_png(grid_time, filename = 'CDN_cluster2_together_cells.png')


In [ ]:
_, cluster_out_conns = fetch_adjacencies(
        sources = NC(
            bodyId = cluster2,
        )
    )

_, cluster_in_conns = fetch_adjacencies(
    sources = NC(
        bodyId = cluster_out_conns['bodyId_post'].unique(),
    ),
    targets = NC(
        bodyId = cluster2,
    )
)

cluster_out_conns

merged_cluster = cluster_out_conns.merge(
    cluster_in_conns,
    left_on='bodyId_post',
    right_on='bodyId_pre',
    suffixes=('_out', '_in'),
)
